In [102]:
-

In [2]:
def ReadFile(inFile):
    with open(inFile, "r") as f:
        content = f.readlines()   
    content = [x.strip() for x in content]
    return (content)

In [ ]:
"""
get opposite of a sentence if conbaina any negation word it check the rest of the sentence to get antonyms of the closest word
ex. I am not so good -> will check every word after no if it has an antonym it wil replace the word and remove "no".
"""
def getAntonyms(word):
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            if l.antonyms():
                return (l.antonyms()[0].name())
    return None

def negateSentence(sentence):
    negative = ['no','not','never','cannot']
    result = []
    wordIndex = 0
    words = sentence.split()
    while wordIndex < len(words):
        stripped = words[wordIndex].strip().lower()
        if ((stripped in negative) or re.match(r".*n't",stripped)):
            for i in range(wordIndex+1,len(words)):
                if (getAntonyms(words[i])):
                    result.append(getAntonyms(words[i]))
                    wordIndex = i
                    break
        else:
            result.append(words[wordIndex])
        wordIndex += 1
    return ' '.join(result)

#negate tweet
def negateTweet(tweet):
    #tweet = Replace_unicoded(tweet)
    result = ''
    delim = '; |, |\*|\n|\.|\?|\:|!'
    sentences = re.split(delim,tweet)
    for sentence in sentences:
        result += negateSentence(sentence)+'.'
    return result

def fixNegation(tweet):
    tweet = negateTweet(tweet)
    return tweet

In [3]:
def ExFeatures(df):
    tweet = df.Body
    df['len'] = [len(t) for t in df.Body]
    def count_URL(tweet):
        URL_count = 0
        for word in tweet.split() :
            if word.startswith('http'):
                URL_count+=1
        return URL_count
    df['URL_count'] = df["Body"].apply(count_URL)
    
    def pos_count(tweet):
        pos_count = 0
        for i in tweet.split():
            if i in positive_Words:
                pos_count+=1
        return pos_count
    df["positive_words_count"] = df["Body"].apply(pos_count)
    
    def neg_count(tweet):
        neg_count = 0
        for i in tweet.split():
            if i in negative_Words:
                neg_count+=1
        return neg_count 
    df["negative_words_count"] = df["Body"].apply(neg_count)
    
    def no_of_dots(tweet):
        liste = [val for c, val in ct.Counter(tweet).items() if c in '.' ]
        if liste == []: liste=[0]
        return liste[0]
    df['no_of_dots'] = df["Body"].apply(no_of_dots)
    
    def no_of_exclmarks(tweet):
        return sum(1 for c in tweet if c in '!' )
    df['no_of_exclmarks'] = df["Body"].apply(no_of_exclmarks)
    
    def no_of_quesmarks(tweet):
        return sum(1 for c in tweet if c in '?' )
    df['no_of_quesmarks'] = df["Body"].apply(no_of_quesmarks)
    
    def no_of_specialcharacters(tweet):
        return sum(1 for c in tweet if c in '$^~*"+=<>%&' )
    df['no_of_specialcharacters'] = df["Body"].apply(no_of_specialcharacters)
    
    def no_of_mentions(tweet):
        return sum(1 for c in tweet if c in '@' )
    df['no_of_mentions'] = df["Body"].apply(no_of_mentions)
    
    def no_of_hashtags(tweet):
        return sum(1 for c in tweet if c in '#' )
    df['no_of_hashtags'] = df["Body"].apply(no_of_hashtags)
    
    def no_of_UpperCase(tweet):
        return sum(1 for c in tweet if c.isupper())
    df['no_of_UpperCase'] = df["Body"].apply(no_of_UpperCase)
    
    def number_of_quotes(tweet):
        return sum(1 for c in tweet if c in '"' )
    df['number_of_quotes'] = df["Body"].apply(number_of_quotes)
    
    def countPosEmoji(tweet):
        numEmoji = 0
        for x in posEmoji:
            if (x[0] in tweet):
                numEmoji += tweet.count(x[0])
        return numEmoji
    df["posEmoji"] = df["Body"].apply(countPosEmoji)    
    
    def countNegEmoji(tweet):
        numEmoji = 0
        for x in negEmoji:
            if (x[0] in tweet):
                numEmoji += tweet.count(x[0])
        return numEmoji
    df["negEmoji"] = df["Body"].apply(countNegEmoji)

    def getPolarity(tweet):
        polarity = TextBlob(tweet).sentiment.polarity
        return polarity
    df["textBlobPolarity"] = df["Body"].apply(getPolarity)

    def getSubjectivity(tweet):
        subjectivity = TextBlob(tweet).sentiment.subjectivity
        return subjectivity
        
    df["textBlobSubjectivity"] = df["Body"].apply(getSubjectivity)   
    
# POS features     
    def is_adjective(tag):
        if tag == 'JJ' or tag == 'JJR' or tag == 'JJS':
            return True
        else:
            return False
    def adjective_count(tweet):
        return sum(is_adjective(i) for i in list(map(lambda x : x[1],pos_tag(tweet.split()))))
    df["adjective_count"] = df["Body"].apply(adjective_count)

    def is_adverb(tag):
        if tag == 'RB' or tag == 'RBR' or tag == 'RBS':
            return True
        else:
            return False
    def adverb_count(tweet):
        return sum(is_adverb(i) for i in list(map(lambda x : x[1],pos_tag(tweet.split()))))
    df["adverb_count"] = df["Body"].apply(adverb_count)

    def is_noun(tag):
        if tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS':
            return True
        else:
            return False
    def noun_count(tweet):
        return sum(is_noun(i) for i in list(map(lambda x : x[1],pos_tag(tweet.split()))))
    df["noun_count"] = df["Body"].apply(noun_count)
    def is_verb(tag):
        if tag == 'VB' or tag == 'VBD' or tag == 'VBG' or tag == 'VBN' or tag == 'VBP' or tag == 'VBZ':
            return True
        else:
            return False
    def verb_count(tweet):
        return sum(is_verb(i) for i in list(map(lambda x : x[1],pos_tag(tweet.split()))))
    df["verb_count"] = df["Body"].apply(verb_count)
    return df

In [4]:
##POS sequence Extraction
def getPOSSeq(tweet,seqlen):
    POS = pos_tag(tweet.split())
    results = []
    for i in range(len(POS)-seqlen+1):
        buffPOS = ''
        for j in range(seqlen):
            buffPOS += POS[i+j][1]
        results.append(buffPOS)
    return(results)

def getTopPOSSequence(tweetslist,seqlen,n_top):
    topNSeqs = []
    POSdict = {'buff':0}
    seqN = 3
    for tweet in tweetslist:
        seqs = getPOSSeq(tweet,seqlen)
        for seq in seqs:
            if seq in POSdict.keys():
                POSdict[seq] += 1
            else:
                POSdict[seq] = 1
    sortedPOSSeq = sorted(POSdict.items(), key=operator.itemgetter(1))[-100:]
    sortedPOSSeqlist = []
    for x in sortedPOSSeq:
        sortedPOSSeqlist.append(x[0])
    return sortedPOSSeqlist

def POSSequenceCount(tweet,seqlen,seq):
    num = getPOSSeq(tweet,seqlen).count(seq)
    if(num):
        return num
    else:
        return 0


In [5]:
# Cleaning the tweets by removing URLs, usernames (mentions) , numbers and special characters 
def clean(tweet,emoticons):
    def Replace_unicoded(tweet):
        soup = BeautifulSoup(tweet, 'lxml')
        souped = soup.get_text()
        souped = ' '.join(re.sub("u2019","'", souped).split())
        souped = ' '.join(re.sub("u002c"," ", souped).split())
        return souped

    tweet = Replace_unicoded(tweet)
    
    def replaceEmoticons(tweet,emoticons):
        for emotion in emoticons:
            tweet = tweet.replace(emotion[0], " "+emotion[1]+" ",100)
        return tweet
    tweet = replaceEmoticons(tweet,emoticons)
    
    def Clean_tweet1(tweet):
        return ' '.join(re.sub("(\w+:\/\/\S+)|(&)|(%)|($)|(@[A-Za-z0-9_]+)|([^0-9A-Za-z_ \t])", "", tweet).split())
    
    tweet = Clean_tweet1(tweet)

    def Remove_numbers(tweet):
        return ' '.join(re.sub("(\s?[0-9]+\.?[0-9]*)", "", tweet).split())
    
    tweet = Remove_numbers(tweet)
    return tweet

In [6]:
positive_Words = ReadFile('positive-words.txt')
negative_Words = ReadFile('negative-words.txt')
posEmoji = [x.split(',') for x in ReadFile('posEmotions.txt')]
negEmoji = [x.split(',') for x in ReadFile('negEmotions.txt')]
emoticons = [x.split(',') for x in ReadFile('emotions.txt')]
positive_Words = positive_Words[35:]
negative_Words = negative_Words[35:]

In [7]:
# reading Dataset
tweets = pd.read_csv("twitter-2015+2013train.csv",names=['ID','Type','Body'])
tweets = tweets[['Type','Body']]
tweets.head()

,Type,Body
0,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,negative,Iranian general says Israel\u2019s Iron Dome c...
4,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...


In [8]:
tweets = ExFeatures(tweets)
tweets.head(3)

,Type,Body,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,...,no_of_UpperCase,number_of_quotes,posEmoji,negEmoji,textBlobPolarity,textBlobSubjectivity,adjective_count,adverb_count,noun_count,verb_count
0,positive,Gas by my house hit $3.39!!!! I\u2019m going t...,70,0,0,0,2,4,0,1,...,5,0,1,0,0.5,1.0,1,0,7,2
1,negative,Theo Walcott is still shit\u002c watch Rafa an...,81,0,0,0,1,0,0,0,...,5,0,0,0,0.0,0.0,1,1,7,1
2,negative,its not that I\u2019m a GSP fan\u002c i just h...,90,0,0,1,2,0,0,0,...,6,0,0,0,-0.8,0.9,0,2,8,2


In [10]:
tweets['Body'] = tweets['Body'].apply(clean,emoticons=emoticons)
tweets.head(3)

,Type,Body,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,...,no_of_UpperCase,number_of_quotes,posEmoji,negEmoji,textBlobPolarity,textBlobSubjectivity,adjective_count,adverb_count,noun_count,verb_count
0,positive,Gas by my house hit Im going to Chapel Hill on...,70,0,0,0,2,4,0,1,...,5,0,1,0,0.5,1.0,1,0,7,2
1,negative,Theo Walcott is still shit watch Rafa and John...,81,0,0,0,1,0,0,0,...,5,0,0,0,0.0,0.0,1,1,7,1
2,negative,its not that Im a GSP fan i just hate Nick Dia...,90,0,0,1,2,0,0,0,...,6,0,0,0,-0.8,0.9,0,2,8,2


In [11]:
test = pd.read_csv('new_english_test.csv',names=['ID','Body'])
test.head(3)

,ID,Body
0,id,tweet
1,218775148495515649,Musical awareness: Great Big Beautiful Tomorro...
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour ana...


In [12]:
test = ExFeatures(test)
test.head(3)

,ID,Body,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,...,no_of_UpperCase,number_of_quotes,posEmoji,negEmoji,textBlobPolarity,textBlobSubjectivity,adjective_count,adverb_count,noun_count,verb_count
0,id,tweet,5,0,0,0,0,0,0,0,...,0,0,0,0,0.0000,0.0000,0,0,1,0
1,218775148495515649,Musical awareness: Great Big Beautiful Tomorro...,87,0,0,0,0,0,0,0,...,6,0,0,0,0.4125,0.4625,1,2,7,3
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour ana...,141,1,0,1,2,0,0,0,...,8,0,0,0,0.0000,0.0000,0,0,13,0


In [13]:
test['Body'] = test['Body'].apply(clean,emoticons=emoticons)
test.head(3)

,ID,Body,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,...,no_of_UpperCase,number_of_quotes,posEmoji,negEmoji,textBlobPolarity,textBlobSubjectivity,adjective_count,adverb_count,noun_count,verb_count
0,id,tweet,5,0,0,0,0,0,0,0,...,0,0,0,0,0.0000,0.0000,0,0,1,0
1,218775148495515649,Musical awareness Great Big Beautiful Tomorrow...,87,0,0,0,0,0,0,0,...,6,0,0,0,0.4125,0.4625,1,2,7,3
2,258965201766998017,On Radiofm Fri Oct Labour analyst Shawn Hattin...,141,1,0,1,2,0,0,0,...,8,0,0,0,0.0000,0.0000,0,0,13,0


In [14]:
seqlen = 2
topPOSSequence = getTopPOSSequence(tweets['Body'].values,seqlen,100)
for seq in topPOSSequence:    
    tweets[seq] = tweets["Body"].apply(POSSequenceCount,seqlen=seqlen,seq=seq)
    test[seq] = test["Body"].apply(POSSequenceCount,seqlen=seqlen,seq=seq)

In [17]:
vectorizer = CountVectorizer(min_df=4,max_df=0.8,ngram_range= (1,2))
vectorized_train = vectorizer.fit_transform(tweets['Body'])
vectorized_trainDF = pd.DataFrame(vectorized_train.toarray(),columns=vectorizer.get_feature_names())
vectorized_trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173 entries, 0 to 10172
Columns: 10242 entries, aa to zulu
dtypes: int64(10242)
memory usage: 794.9 MB


In [18]:
vectorized_test = vectorizer.transform(test['Body'])
vectorized_testDF = pd.DataFrame(vectorized_test.toarray(),columns=vectorizer.get_feature_names())
vectorized_testDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036 entries, 0 to 3035
Columns: 10242 entries, aa to zulu
dtypes: int64(10242)
memory usage: 237.2 MB


In [19]:
final_train = pd.concat([tweets['Type'],tweets.drop(['Type','Body'],axis=1) ,vectorized_trainDF],axis=1)
final_train.head()

,Type,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,no_of_mentions,...,zap,zap all,zap catch,zap nba,zimmerman,zombie,zombies,zone,zuckerman,zulu
0,positive,70,0,0,0,2,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,81,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,negative,90,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,negative,135,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,neutral,143,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
final_test = pd.concat([test.drop(['Body'],axis=1) ,vectorized_testDF],axis=1)
final_test.head()

,ID,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,no_of_mentions,...,zap,zap all,zap catch,zap nba,zimmerman,zombie,zombies,zone,zuckerman,zulu
0,id,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,218775148495515649,87,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,258965201766998017,141,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,262926411352903682,141,0,1,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,254948834910818305,132,0,0,0,1,0,0,3,2,...,0,0,0,0,0,0,0,0,0,0


## Word Embedding

In [21]:
#glove2word2vec(glove_input_file="E:/glove.twitter.27B.200d.txt", word2vec_output_file="glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("glove_vectors.txt", binary=False)

In [22]:
def featureVec(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float64")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating the average feature vector
def getAvgFeatureVecs(tweets, model, num_features):
    counter = 0
    tweetFeatureVecs = np.zeros((len(tweets),num_features),dtype="float64")
    for tweet in tweets:  
        if counter%1000. == 0.:
            print ("Tweet %d of %d" % (counter, len(tweets)))
        tweetFeatureVecs[counter] = featureVec(tweet, model, num_features)
        counter = counter+1
        
    return tweetFeatureVecs

In [23]:
trainDataVecs = getAvgFeatureVecs(tweets['Body'], glove_model, 200)

Tweet 0 of 10173


C:\Users\be231\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\be231\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: invalid value encountered in true_divide


Tweet 1000 of 10173
Tweet 2000 of 10173
Tweet 3000 of 10173
Tweet 4000 of 10173
Tweet 5000 of 10173
Tweet 6000 of 10173
Tweet 7000 of 10173
Tweet 8000 of 10173
Tweet 9000 of 10173
Tweet 10000 of 10173


In [24]:
testDataVecs = getAvgFeatureVecs(test['Body'], glove_model, 200)

Tweet 0 of 3036


C:\Users\be231\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Tweet 1000 of 3036
Tweet 2000 of 3036
Tweet 3000 of 3036


In [25]:
trainDataVecsDF=pd.DataFrame(trainDataVecs,columns = ['W2vFeature %s_' %i for i in range(trainDataVecs.shape[1])])
trainDataVecsDF.shape

(10173, 200)

In [26]:
testDataVecsDF=pd.DataFrame(testDataVecs,columns = ['W2vFeature %s_' %i for i in range(testDataVecs.shape[1])])
testDataVecsDF.shape

(3036, 200)

In [120]:
bow_w2v = pd.concat([final_train, trainDataVecsDF], axis=1)
bow_w2v.shape

#import collections
#duplicate = [item for item, count in collections.Counter(rrrr).items() if count > 1]
#bow_w2v.drop(duplicate,axis=1)
#rrrr = list(bow_w2v)
#print (len([item for item, count in collections.Counter(rrrr).items() if count > 1]))

0


In [121]:
bow_w2v_test = pd.concat([final_test,testDataVecsDF], axis=1)
bow_w2v_test.shape

(3036, 10563)

In [126]:
bow_w2vcopy = bow_w2v.copy()
bow_w2v_testcopy = bow_w2v_test.copy()
bow_w2vcopy.fillna(0, inplace=True)
bow_w2v_testcopy.fillna(0, inplace=True)

In [ ]:
#XGBoost Grid search
tweets_train = final_train.drop(["Type"],axis=1)
train_labels = final_train['Type']
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([('classifier', XGBClassifier())])
search_space = [{'classifier': [XGBClassifier()],
                 'classifier__nthread': [-1],
                 'classifier__n_estimators': [103,201, 403],
                 'classifier__max_depth': [3,10,15, 30],
                 'classifier__objective': ["multi:softmax","binary:logistic"],
                 'classifier__learning_rate': [0.05, 0.1, 0.15, 0.3]}]


clf_gss = GridSearchCV(pipe, search_space, cv=5, verbose=0)

best_model = clf_gss.fit(final_train.drop(["Type"],axis=1), final_train['Type'])

# View best model
print (best_model.best_estimator_.get_params()['classifier'])

In [ ]:
#Random Forest
pipe = Pipeline([('classifier', RandomForestClassifier())])
search_space = [{'classifier': [RandomForestClassifier()],
                 'classifier__n_estimators': [10, 100, 1000],
                 'classifier__max_features': [1, 2, 3]}]
               

clf_gss = GridSearchCV(pipe, search_space, cv=5, verbose=0)

best_model = clf_gss.fit(final_train.drop(["Type"],axis=1), final_train['Type'])

# View best model
print (best_model.best_estimator_.get_params()['classifier'])

In [ ]:
#Classifiers trails
clfrKnearest = neighbors.KNeighborsClassifier(n_neighbors=3)
clfrKnearest.fit(tweets_train,train_labels)
predictedK = clfr.predict(tweets_val)
acc = metrics.accuracy_score(val_labels,predictedK) 
print('accuracy = ',acc*100,'%')

clfrNaive = naive_bayes.MultinomialNB()
clfrNaive.fit(tweets_train,train_labels)
predictedNaive = clfr.predict(tweets_val)
acc = metrics.accuracy_score(val_labels,predictedNaive)
print('accuracy = ',acc*100,'%')

for i in range(len(val_labels)):
    print (val_labels[i],predictedNaive[i],predictedXGB[i])
acc = metrics.accuracy_score(val_labels,predicted)  
print('accuracy = ',acc*100,'%')

In [131]:
clf_gs = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

clf_gs = XGBClassifier(nthread=-1,n_estimators=201,max_depth=20,objective="multi:softmax",learning_rate=.12)
clf_gs.fit(bow_w2vcopy.drop(["Type"],axis=1), bow_w2vcopy['Type'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.12, max_delta_step=0,
       max_depth=20, min_child_weight=1, missing=None, n_estimators=201,
       n_jobs=1, nthread=-1, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [132]:
bow_w2vcopy['Type'].head()

0    positive
1    negative
2    negative
3    negative
4     neutral
Name: Type, dtype: object

In [133]:
bow_w2v_testcopy.head()

,ID,len,URL_count,positive_words_count,negative_words_count,no_of_dots,no_of_exclmarks,no_of_quesmarks,no_of_specialcharacters,no_of_mentions,...,W2vFeature 190_,W2vFeature 191_,W2vFeature 192_,W2vFeature 193_,W2vFeature 194_,W2vFeature 195_,W2vFeature 196_,W2vFeature 197_,W2vFeature 198_,W2vFeature 199_
0,id,5,0,0,0,0,0,0,0,0,...,-0.070661,0.377460,-0.579102,0.296534,-0.099056,0.135564,0.030092,-0.263439,0.034199,-0.031498
1,218775148495515649,87,0,0,0,0,0,0,0,0,...,-0.177638,0.240933,-0.166556,0.216743,-0.061036,0.097181,-0.108041,-0.370387,0.190655,0.089424
2,258965201766998017,141,1,0,1,2,0,0,0,0,...,-0.135459,0.225265,-0.154029,0.163532,-0.097447,0.072990,-0.142078,-0.415543,0.156611,0.135041
3,262926411352903682,141,0,1,0,3,0,0,0,0,...,-0.135555,0.204452,-0.138231,0.239701,-0.057297,0.037440,-0.106794,-0.405302,0.187243,0.113300
4,254948834910818305,132,0,0,0,1,0,0,3,2,...,-0.100995,0.239441,-0.167882,0.224557,-0.017336,0.045128,-0.105315,-0.407548,0.186473,0.125416


In [134]:
predicted = clf_gs.predict(bow_w2v_testcopy.drop(["ID"],axis=1))
predicted = predicted[1:]

In [ ]:
results = pd.read_csv('new_english_test.csv')
results = results.assign(sentiment=predicted)
results.drop(['tweet'], axis=1)
results.to_csv(path_or_buf="test.csv",columns=['id','sentiment'],index=False)